# Seq2seq进行机器翻译与存取模型

当前最复杂的NLP应用之一就是机器翻译，我们今天来讲一个使用Seq2seq模型进行机器翻译的例子，顺便给大家介绍一下存取模型。

In [1]:
import torch
import numpy as np
from torch import nn

我们本次就不用emb啦，因为我们今天的任务是英语翻译成法语，而英语的emb处理起来要花的时间太多了，就直接初始化吧。

首先我们来建立一个Seq2seq的网络。一般的Seq2seq网络都主要分成两个部分：Encoder和Decoder。其中Encoder是解码器，负责将一个句子转化成一个张量表示。

In [6]:
# Encoder部分，其实就是一个标准的RNN网络
class EncoderRNN(nn.Module):
    # 这里的input_size其实就是英语的词表大小，hidden_size是超参
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        # 词嵌入层，这里没有初始化，就让它随着训练自己计算吧
        self.embedding = nn.Embedding(input_size, hidden_size)
        # 注意这里用了batch_first，所以接收的输入是(batch_size, seq_length, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.gru(embedded)
        return output, hidden

Decoder是解码器，用于接收Encoder的张量表示，然后通过一定的方法依次输出需要的序列。

In [7]:
# Decoder部分，我们这里先用标准的RNN网络，实际上现在大部分是使用带Attention的RNN网络
class DecoderRNN(nn.Module):
    # 这里的output_size其实就是法语的词表大小，hidden_size必须要和刚才Encoder的hidden_size一致
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        # 注意这里用了batch_first，所以接收的输入是(batch_size, seq_length, hidden_size)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, batch_first=True)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, hidden):
        # 输入的x是(batch_size)大小，经过embedding后变成(batch_size, hidden_size)，但还是和GRU的要求不一致
        # 好在decoder中，seq_length始终是1，所以我们只需要用unsqueeze函数，在中间加一维即可
        # (batch_size, hidden_size) -> (batch_size, 1, hidden_size) -> (batch_size, seq_length=1, hidden_size)
        embedded = self.dropout(self.embedding(x).unsqueeze(1))
        # 通过RNN，得到下一个输出，以及输出对应的hidden_state
        output, hidden = self.gru(embedded, hidden)
        # 用fc输出层进行预测
        prediction = self.out(output)
        return prediction, hidden

最后我们建立一个完整的Seq2seq网络，包含Encoder，Decoder和一些辅助的函数/层。
![avatar](素材/seq2seq.png)

In [8]:
class Seq2seq_translater(nn.Module):
    # input_size: 英文词表数
    # hidden_size: 超参
    # output_size: 法文词表数
    # seq_length: 最长序列长度
    def __init__(self, input_size, hidden_size, output_size, seq_length):
        super().__init__()
        # 用上之前的Encoder
        self.encoder = EncoderRNN(input_size, hidden_size)
        # 用上之前的Decoder
        self.decoder = DecoderRNN(hidden_size, output_size)
        # 小小的注意事项：hidden_size，num_layers必须一致，不然报错
        # 用交叉熵损失函数
        self.loss_fct = nn.CrossEntropyLoss()
        # 这两个属性存下来，还要用到
        self.output_size = output_size
        self.seq_length = seq_length

    def forward(self, x, y=None):
        # 别忘记了我们在GRU里使用了batch_first，所以输入和输出的shape都是batch在最前面的
        # output是(batch_size, seq_length, hidden_size)
        # LSTM -> output, (hidden_stat, cell_state)
        # 但是hidden(以及如果你用LSTM，还多一个cell)，还是原来的样子
        # hidden是(num_layers*num_directions, batch_size, hidden_size)
        output, hidden = self.encoder(x)
        # 用BOS(BEGIN OF SENTENCE)来做Decoder的初始输入，我们可以直接从输入中提取
        # 每一轮的输入的shape就是个(batch_size)，一维的
        decoder_input = x[:, 0]
        # 依次存下每轮的输出
        outputs = []
        # 一轮一轮地进行迭代
        for i in range(self.seq_length):
            # decoder迭代一次
            output, hidden = self.decoder(decoder_input, hidden)
            # 存在outputs里面
            outputs.append(output)
            # 这个时候的output是(batch_size, 1, output_size)，我们在最后一维上做argmax，就能得到输出的结果
            # 但是别忘记了，输入是(batch_size)，所以我们需要进行一个squeeze，把当中那个1去了
            pred = output.squeeze(dim=1).argmax(dim=-1)
            # 下一轮的输入就是本轮的预测
            decoder_input = pred
        # 最后我们把所有预测的连起来，在当中那维连起来
        # (batch_size, 1, output_size) -> (batch_size, seq_length, output_size)
        total_output = torch.cat(outputs, dim=1)
        # 还是一样，如果有y就输出loss，没有y就输出预测
        if y is not None:
            return self.loss_fct(total_output.view(-1, self.output_size), y.view(-1))
        else:
            return total_output.squeeze()

但是有没有发现，我们需要初始化模型的话，还需要俩参数——英语词表大小和法语词表大小，这个我们没法直接确定，需要先读语料库才行。Tokenizer和data_utils已经基于这个notebook进行更新啦，加入了一些新功能~

In [9]:
# 针对英语翻译法语的语料库的dataset_readers
from dataset_readers.trans import *
# Tokenizer进行了更新，加入了normlizeString和get_vocabs两个类方法，不需要实例即可使用
from utils.tokenizer import Tokenizer
from torch.utils.data import TensorDataset, DataLoader

def load_data(seq_length, batch_size):
    # 实例化一个readers
    data_loader = En2Fr_Trans()
    # 获取训练语料
    train_examples = data_loader.get_train_examples()
    # 英语的数据
    en_lines = [Tokenizer.normalizeString(i.text_a) for i in train_examples]
    # 法语的数据
    fr_lines = [Tokenizer.normalizeString(i.text_b) for i in train_examples]
    
    # 我们需要两个tokenizer，一个用于英语一个用于法语
    tokenizer_a = Tokenizer(Tokenizer.get_vocabs(en_lines))
    tokenizer_b = Tokenizer(Tokenizer.get_vocabs(fr_lines))
    # 得到英语和法语词表大小，现在就可以初始化model啦
    word_cnt_a = len(tokenizer_a.vocab)
    word_cnt_b = len(tokenizer_b.vocab)
    
    # 但还是让我们先把数据读完吧，这个函数用于将examples转换成features，然后再生成DataLoader
    def generate_dataloader(examples, tokenizer_a, tokenizer_b, seq_length):
        features = convert_sents_pair(examples, tokenizer_a, tokenizer_b, seq_length)
        text_a = torch.tensor([f.text_a for f in features], dtype=torch.long)
        text_b = torch.tensor([f.text_b for f in features], dtype=torch.long)
        dataset = TensorDataset(text_a, text_b)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return dataloader
    # 生成DataLoader
    train_dataloader = generate_dataloader(train_examples, tokenizer_a, tokenizer_b, seq_length)
    return train_dataloader, word_cnt_a, word_cnt_b, tokenizer_a, tokenizer_b

获取训练的数据集，同时得到了word_cnt之后就能初始化网络了。

In [10]:
# 我们做机器翻译啦，这个时候序列长度就需要比较长了，作为实验我们先使用64
seq_length = 64
# 我在预训练的过程中，batch_size是64，大家可以视自己情况进行调整
batch_size = 32
train_dataloader, word_cnt_a, word_cnt_b, tokenizer_en, tokenizer_fr = load_data(seq_length, batch_size)
# 超参hidden_state设为256，只是随手设置的，你也可以自己调整
model = Seq2seq_translater(word_cnt_a, 256, word_cnt_b, seq_length)
# 我之前在常老师的服务器上跑了一遍，我们来读取模型
# model.save() -> model.load()
# torch.save(model.state_dict()) -> model.load_state_dict(torch.load())
# ELECTRA -> BERT 上面的就不行了，下面的可以
model.load_state_dict(torch.load('./models/seq2seq_translate.bin'))

if torch.cuda.is_available():
    model.to(torch.device('cuda'))
# 使用print可以打印出网络的结构
print(model)

Seq2seq_translater(
  (encoder): EncoderRNN(
    (embedding): Embedding(13043, 256)
    (gru): GRU(256, 256, batch_first=True)
  )
  (decoder): DecoderRNN(
    (embedding): Embedding(21334, 256)
    (dropout): Dropout(p=0.1, inplace=False)
    (gru): GRU(256, 256, batch_first=True)
    (out): Linear(in_features=256, out_features=21334, bias=True)
  )
  (loss_fct): CrossEntropyLoss()
)


依然使用Adam优化器。

In [11]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=0.0001)
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)


开始训练。这里就不运行训练了，因为这个一个epoch估计是10分钟左右了。

In [ ]:
import time

def train(model, optimizer, train_dataloader, epoch=5):
    total_start_time = time.time()
    for i in range(epoch):
        epoch_start_time = time.time()
        print("epoch %d/%d" % (i + 1, epoch))
        model.train()
        total_loss = []
        for text_a, text_b in train_dataloader:
            if torch.cuda.is_available():
                text_a = text_a.to(torch.device('cuda'))
                text_b = text_b.to(torch.device('cuda'))
            optimizer.zero_grad()
            loss = model(text_a, text_b)
            total_loss.append(loss.item())
            loss.backward()
            optimizer.step()
        print("epoch: %d, loss: %.6f" % (i + 1, sum(total_loss) / len(total_loss)))
        epoch_end_time = time.time()
        print("epoch time: %d s" % (epoch_end_time - epoch_start_time))
        torch.save(model.state_dict(), './models/seq2seq_translate.bin')
    total_end_time = time.time()
    print("total time: %d s" % (total_end_time - total_start_time))

epoch = 1
train(model, optimizer, train_dataloader, epoch)

最后做一下测试。

In [12]:
vocab = tokenizer_fr.vocab
id2word = {v: k for k,v in vocab.items()}

def tensor2text(pred):
    pred = pred.detach().cpu().numpy()
    pred = np.argmax(pred, axis=-1)
    output = []
    for i in pred:
        w = id2word[i]
        if w == 'EOS':
            break
        output.append(w)
    return ' '.join(output)

while True:
    s = input()
    if s == 'quit':
        break
    s = [sents_pair_example(s, '')]
    s = convert_sents_pair(s, tokenizer_en, tokenizer_fr, seq_length)
    text_a = torch.tensor([f.text_a for f in s], dtype=torch.long)
    with torch.no_grad():
        if torch.cuda.is_available():
            text_a = text_a.to(torch.device('cuda'))
        text_b = model(text_a)
        print(tensor2text(text_b))
        

Good afternoon.
lavee nos arriverai recalee ferree
convert
efficacement maturite segment
quit


看起来不太行……主要是网络结构，数据，训练时间啥的都不太够，就当做给大家的一个参考吧。